In [1]:
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
#get final crop data into a dataframe

final_crop_data = pd.DataFrame(pd.read_csv('Resources/final_crop_data.csv'))
# final_crop_data = final_crop_data.drop(columns=["Unnamed: 0"])
final_crop_data

,Unnamed: 0,Area,Crop,Year,Crop_Yield,Pesticide_hg/ha,Rain_mm,Temp_c
0,3,Albania,Sorghum,1990,12500,121.00,1485.0,16.37
1,5,Albania,Wheat,1990,30197,121.00,1485.0,16.37
2,9,Albania,Sorghum,1991,6667,121.00,1485.0,15.36
3,11,Albania,Wheat,1991,20698,121.00,1485.0,15.36
4,15,Albania,Sorghum,1992,3747,121.00,1485.0,16.06
...,...,...,...,...,...,...,...,...
8938,28236,Zimbabwe,Sorghum,2012,2046,3375.53,657.0,20.52
8939,28239,Zimbabwe,Wheat,2012,24420,3375.53,657.0,20.52
8940,28240,Zimbabwe,Cassava,2013,46000,2550.07,657.0,19.76
8941,28244,Zimbabwe,Sorghum,2013,3066,2550.07,657.0,19.76


In [3]:
#find unique crops

final_crop_data['Crop'].unique()

array(['Sorghum', 'Wheat', 'Cassava'], dtype=object)

In [9]:
#get data for Sorghum
sorghum_df = final_crop_data[final_crop_data['Crop'] == 'Sorghum'].copy()

#get data for Wheat
wheat_df = final_crop_data[final_crop_data['Crop'] == 'Wheat'].copy()

#get data for Cassava
cassava_df = final_crop_data[final_crop_data['Crop'] == 'Cassava'].copy()

# to get all crops into one dataframe
crops_of_interest = ['Maize', 'Potatoes', 'Rice, paddy', 'Sorghum', 'Soybeans', 'Wheat','Cassava', 'Sweet potatoes', 'Plantains and others', 'Yams']
combined_crop_df = final_crop_data[final_crop_data['Crop'].isin(crops_of_interest)]
combined_crop_df.to_csv("Resources/final_crop_data.csv", index=False)
combined_crop_df
                                


,Unnamed: 0,Area,Crop,Year,Crop_Yield,Pesticide_hg/ha,Rain_mm,Temp_c
0,3,Albania,Sorghum,1990,12500,121.00,1485.0,16.37
1,5,Albania,Wheat,1990,30197,121.00,1485.0,16.37
2,9,Albania,Sorghum,1991,6667,121.00,1485.0,15.36
3,11,Albania,Wheat,1991,20698,121.00,1485.0,15.36
4,15,Albania,Sorghum,1992,3747,121.00,1485.0,16.06
...,...,...,...,...,...,...,...,...
8938,28236,Zimbabwe,Sorghum,2012,2046,3375.53,657.0,20.52
8939,28239,Zimbabwe,Wheat,2012,24420,3375.53,657.0,20.52
8940,28240,Zimbabwe,Cassava,2013,46000,2550.07,657.0,19.76
8941,28244,Zimbabwe,Sorghum,2013,3066,2550.07,657.0,19.76


## this is to show just one crop at a time

In [5]:
fig_sorghum = px.choropleth(
    sorghum_df,
    locations="Area",
    locationmode="country names",
    color="Crop_Yield",
    hover_name="Area",
    animation_frame="Year",
    color_continuous_scale="YlGnBu",
    title="Sorghum Yield by Country Over Time",
    labels={"Crop_Yield": "Yield (hg/ha)"}
)

# Update layout for better appearance
fig_sorghum.update_layout(
    geo=dict(showframe=False, showcoastlines=False),
    coloraxis_colorbar=dict(title="Yield (hg/ha)")
)

fig_sorghum.show()

## This is to show all three crops at once, with separate maps.

This uses the same scale for all, and Cassava is the largest.

In [6]:
# Create animated choropleth
fig = px.choropleth(
    combined_crop_df,
    locations="Area",
    locationmode="country names",
    color="Crop_Yield",
    hover_name="Area",
    animation_frame="Year",
    color_continuous_scale="YlGnBu",
    facet_col="Crop",  # Split by crop
    title="Crop Yield Over Time",
    labels={"Crop_Yield": "Yield (hg/ha)"}
)

fig.update_layout(
    geo=dict(showframe=False, showcoastlines=False),
    coloraxis_colorbar=dict(title="Yield (hg/ha)")
)

fig.show()

## This is a working mapped graph allowing to choose the crop and show the yield for each year for each country

This does not go through all the years automatically, which would be nice.

In [7]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Load and clean data
df = pd.read_csv("final_crop_data.csv")
df["Rain_mm"] = pd.to_numeric(df["Rain_mm"], errors='coerce')

# it is not finding the unnamed column, so I am removing the below line:
# df = df.drop(columns=["Unnamed: 0"])

# Filter to crops of interest
crops = ['Maize', 'Potatoes', 'Rice, paddy', 'Sorghum', 'Soybeans', 'Wheat','Cassava', 'Sweet potatoes', 'Plantains and others', 'Yams']
df = df[df["Crop"].isin(crops)]

# Create the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Crop Yield by Country Over Time", style={"textAlign": "center"}),

    html.Div([
        html.Label("Select Crop:"),
        dcc.Dropdown(
            id='crop-dropdown',
            options=[{'label': crop, 'value': crop} for crop in crops],
            value='Sorghum',
            clearable=False
        ),
    ], style={"width": "30%", "display": "inline-block", "padding": "10px"}),

    html.Div([
        html.Label("Select Year:"),
        dcc.Slider(
            id='year-slider',
            min=df['Year'].min(),
            max=df['Year'].max(),
            step=1,
            value=df['Year'].min(),
            marks={str(year): str(year) for year in sorted(df['Year'].unique())},
            tooltip={"placement": "bottom", "always_visible": False}
        ),
    ], style={"width": "60%", "padding": "20px"}),

    dcc.Graph(id='yield-map', style={"height": "700px"})
])

@app.callback(
    Output('yield-map', 'figure'),
    Input('crop-dropdown', 'value'),
    Input('year-slider', 'value')
)
def update_map(selected_crop, selected_year):
    filtered_df = df[(df["Crop"] == selected_crop) & (df["Year"] == selected_year)]

    fig = px.choropleth(
        filtered_df,
        locations="Area",
        locationmode="country names",
        color="Crop_Yield",
        hover_name="Area",
        color_continuous_scale="YlGnBu",
        labels={"Crop_Yield": "Yield (hg/ha)"},
        title=f"{selected_crop} Yield in {selected_year}"
    )

    fig.update_layout(
        geo=dict(showframe=False, showcoastlines=False),
        coloraxis_colorbar=dict(title="Yield (hg/ha)")
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=0)

AssertionError: Expecting an integer from 1 to 65535, found port=0